In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta

In [2]:
url_template = 'https://tenki.jp/past/{}/{:02d}/{:02d}/weather/3/23/'

In [3]:
url = url_template.format(2019, 1, 1) #URL to get the data on 2019/1/1
html_doc = requests.get(url).text

In [4]:
soup = BeautifulSoup(html_doc)
#soup.prettify()

In [5]:
table = soup.find(lambda tag: tag.name == 'table' and 'past-live-area-pref-list-entries' in tag
                 ['class'])

In [6]:
df = pd.read_html(str(table), na_values=['---'], header=0)[0]

In [7]:
df #to check how is going 

,地点名,天気,天気.1,最高気温,最低気温,日積算降水量(mm),日の出,日の入
0,長野,NaN,晴のち雪,5.6℃,-5.6℃,0.5mm,06:59,16:42
1,松本,NaN,不明,7.0℃,-7.9℃,0.0mm,06:59,16:44
2,諏訪,NaN,不明,5.1℃,-8.5℃,0.0mm,NaN,NaN
3,飯田,NaN,不明,8.1℃,-7.3℃,0.0mm,NaN,NaN
4,軽井沢,NaN,不明,3.7℃,-10.0℃,0.0mm,NaN,NaN


In [8]:
df.drop(df.shape[0] - 1, inplace =True)
df.columns = ['city', 'weather', 'weather1', 'max_temp', 'min_temp', 'rain', 'sunrise', 
              'sunset']
df['date'] = pd.to_datetime('{}/{}/{}'.format(2019, 1, 1))

In [9]:
df #to check how is going 

,city,weather,weather1,max_temp,min_temp,rain,sunrise,sunset,date
0,長野,NaN,晴のち雪,5.6℃,-5.6℃,0.5mm,06:59,16:42,2019-01-01
1,松本,NaN,不明,7.0℃,-7.9℃,0.0mm,06:59,16:44,2019-01-01
2,諏訪,NaN,不明,5.1℃,-8.5℃,0.0mm,NaN,NaN,2019-01-01
3,飯田,NaN,不明,8.1℃,-7.3℃,0.0mm,NaN,NaN,2019-01-01


In [10]:
#to automation
def get_weather_for_one_day(year, month, day):
    url = url_template.format(year, month, day)
    html_doc = requests.get(url).text
    soup = BeautifulSoup(html_doc)
    table = soup.find(lambda tag: tag.name == 'table' and 'past-live-area-pref-list-entries'
                     in tag ['class'])
    
    try:
        df = pd.read_html(str(table), na_values=['---'], header=0)[0]
    except Exception as e:
        print(e)
        raise ValueError('No data is available for {}/{:02d}/{:02d}'.format(year, month, day))
    df.drop(df.shape[0] - 1, inplace=True)
    df.columns = ['city', 'weather', 'weather1', 'max_temp', 'min_temp', 'rain', 'sunrise', 
              'sunset']
    df['date'] = pd.to_datetime('{}/{}/{}'.format(year, month, day))
    
    return df

In [11]:
#to download  the weather data from a specific date
def get_weather_from_date(year, month, day):
    date = datetime(year, month, day)
    today = datetime.utcnow()
    df = []
    while (today - date).days >= 1:
        try:
            df.append(get_weather_for_one_day(date.year, date.month, date.day))
        except ValueError as e:
            print(e)
        finally:
            date += timedelta (days=1)
    return pd.concat(df)

In [12]:
df = get_weather_from_date(2019, 1, 1)

In [13]:
dfc = df.copy

In [14]:
df

,city,weather,weather1,max_temp,min_temp,rain,sunrise,sunset,date
0,長野,NaN,晴のち雪,5.6℃,-5.6℃,0.5mm,06:59,16:42,2019-01-01
1,松本,NaN,不明,7.0℃,-7.9℃,0.0mm,06:59,16:44,2019-01-01
2,諏訪,NaN,不明,5.1℃,-8.5℃,0.0mm,NaN,NaN,2019-01-01
3,飯田,NaN,不明,8.1℃,-7.3℃,0.0mm,NaN,NaN,2019-01-01
0,長野,NaN,雪のち晴,3.6℃,-1.1℃,3.0mm,07:00,16:43,2019-01-02
...,...,...,...,...,...,...,...,...,...
3,飯田,NaN,晴,18.0℃,4.6℃,0.0mm,NaN,NaN,2020-04-09
0,長野,NaN,晴一時雨,10.6℃,1.0℃,0.0mm,05:20,18:17,2020-04-10
1,松本,NaN,晴,11.7℃,0.0℃,0.0mm,05:22,18:18,2020-04-10
2,諏訪,NaN,晴,10.6℃,2.3℃,0.0mm,NaN,NaN,2020-04-10


In [15]:
#To clean the data 
def clean_data(df, city_mapping):
    df = df.copy()
    df.city = df.city.map(city_mapping) 
    df.weather1 = df.weather1.map(weather1_mapping) 
    df = df.fillna('Unknown')
    df = df.drop(['weather'], axis = 1) 
    df = df[df.city == 'Matsumoto']
    df = df.reset_index(drop=True)
    start_date = '2019-01-01'
    end_date = '2019-12-31'
    mask = (df['date'] >= start_date) & (df['date'] <= end_date)
    df = df.loc[mask]
    
    return df

In [16]:
#To translate the main cities in Nagano prefecture
city_mapping = {
    '長野': 'Nagano',
    '松本': 'Matsumoto',
    '諏訪': 'Suwa',
    '飯田': 'Iida'
}

weather1_mapping = {
    '不明': 'unknown', 
    '雪': 'snow', 
    '雪のち雨': 'snow and then rain', 
    '雪一時雨': 'sleet', 
    '雨': 'rain', 
    '晴': 'sunny', 
    '曇のち雨': 'cloudy then rain', 
    '晴のち曇': 'cloudy weather', 
    '曇のち晴': 'cloudy then sunny',
    '晴一時雪': 'fine snow', 
    '雨のち晴': 'rain then sunny', 
    '曇': 'cloudy', 
    '雪のち晴': 'snow then sunny', 
    '雨のち曇': 'cloudy after rain', 
    '曇時々晴': 'partly cloudy', 
    '雪のち曇': 'cloudy after snow', 
    '晴一時雨': 'sunny partly rain',
    '晴のち雨': 'sunny after rain'
}

In [17]:
cleaned_df = clean_data(df, city_mapping)

In [23]:
cleaned_df.head(10)

,city,weather1,max_temp,min_temp,rain,sunrise,sunset,date
0,Matsumoto,unknown,7.0℃,-7.9℃,0.0mm,06:59,16:44,2019-01-01
1,Matsumoto,unknown,4.4℃,-0.7℃,1.5mm,06:59,16:44,2019-01-02
2,Matsumoto,unknown,4.3℃,-6.0℃,0.0mm,06:59,16:45,2019-01-03
3,Matsumoto,unknown,4.7℃,-8.2℃,0.0mm,07:00,16:46,2019-01-04
4,Matsumoto,unknown,10.4℃,-2.4℃,0.0mm,07:00,16:47,2019-01-05
5,Matsumoto,unknown,1.3℃,-2.7℃,0.0mm,07:00,16:48,2019-01-06
6,Matsumoto,unknown,4.9℃,-7.3℃,0.0mm,07:00,16:49,2019-01-07
7,Matsumoto,unknown,8.8℃,-5.3℃,0.0mm,07:00,16:50,2019-01-08
8,Matsumoto,unknown,2.4℃,-2.2℃,0.0mm,07:00,16:50,2019-01-09
9,Matsumoto,unknown,1.5℃,-8.9℃,0.0mm,07:00,16:51,2019-01-10


In [24]:
cleaned_df.to_csv('weather.csv')

In [20]:
cleaned_df.isnull().sum()

city        0
weather1    0
max_temp    0
min_temp    0
rain        0
sunrise     0
sunset      0
date        0
dtype: int64

In [21]:
cleaned_df.isna().sum()

city        0
weather1    0
max_temp    0
min_temp    0
rain        0
sunrise     0
sunset      0
date        0
dtype: int64

In [22]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365 entries, 0 to 364
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   city      365 non-null    object        
 1   weather1  365 non-null    object        
 2   max_temp  365 non-null    object        
 3   min_temp  365 non-null    object        
 4   rain      365 non-null    object        
 5   sunrise   365 non-null    object        
 6   sunset    365 non-null    object        
 7   date      365 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 25.7+ KB
